# Install Nginx

Eine Anleitung findet man hier: https://www.raspberrypi.org/documentation/remote-access/web-server/nginx.md

Zunächst die Paketdatenbank aktualisieren. Danach das Paket nginx installieren. Die Option `--yes` kann auch durch `-y` ersetzt werden.

In [ ]:
sudo apt-get update
sudo apt-get install --yes nginx

Nun kann man auf http://pi-sense  (_pi-sense_ durch eigenen Hostname ersetzen) die Nginx Webseite besucht werden.

In [ ]:
cat /var/www/html/index.nginx-debian.html

In [ ]:
sudo sed -i 's/to nginx/to pi-sense/g' /var/www/html/index.nginx-debian.html

In [ ]:
sudo sed -i 's/Debian/Respbian/g' /var/www/html/index.nginx-debian.html

In [ ]:
sudo sed -i 's/debian/respbian/g' /var/www/html/index.nginx-debian.html

In [ ]:
sudo sed -i 's/respbian.org/debian.org/g' /var/www/html/index.nginx-debian.html

## Konfiguration von nginx

In [ ]:
cat /etc/nginx/nginx.conf

Für uns ist zunächst die Zeilen `include /etc/nginx/conf.d/*.conf;` und `include /etc/nginx/sites-enabled/*;` wichtig.

Mit `include /etc/nginx/conf.d/*.conf;` werden alle Dateien mit der Endung __.conf__ im Pfad __/etc/nginx/conf.d/__ inkludiert.

Mit `include /etc/nginx/sites-enabled/*;` werden alle Dateien aus dem Ordner __/etc/nginx/sites-enabled__ inkludiert.

In [ ]:
ls -lachi /etc/nginx/conf.d

In [ ]:
ls -lachi /etc/nginx/sites-enabled

Hier finden wir einen Link zur Datei `/etc/nginx/sites-available/default`

In [ ]:
cat -n /etc/nginx/sites-available/default

Nach http://serverfault.com/questions/312494/hosting-folder-in-the-home-directory-using-nginx soll folgender Block im Abschnitt server eingefügt werden.

```
location ~ ^/~(.+?)(/.*)?$ {
    alias /home/$1/www$2;
    autoindex on;
}
```

Dies können wir mit dem Editor erledigen.

### Sed Ausflug

Hier das Einfügen mit `sed` und grep.

Das Suchen nach der Zeichenkette __#}__

__Achtung__ Das Zeichen # muss demaskiert werden

In [ ]:
grep -n \#} /etc/nginx/sites-available/default

Mit `grep` kann man nach einem Suchstring (RegEx) in einer Datei gesucht werden. Die Option `-n` gibt die Zeilen nummer aus.

In [ ]:
grep -n $'\t#}' /etc/nginx/sites-available/default

Ein Hack um nach Tabulatoren zu suchen. Siehe hier: http://stackoverflow.com/questions/1825552/grep-a-tab-in-unix

Das Kommando `sed` ohne die Option `-i` gibt die Datei auf der Konsole aus, ohne diese zu verändern.

In [ ]:
sed 's/\t#}/foo/' /etc/nginx/sites-available/default

Erwischt also alle zwei Vorkommen.

In [ ]:
sudo sed '0,/\t#}/s/\t#}/foo/' /etc/nginx/sites-available/default

Erwischt erstes Vorkommen (Informatiker zählen ab 0). Siehe hier: http://stackoverflow.com/questions/148451/how-to-use-sed-to-replace-only-the-first-occurrence-in-a-file

In [ ]:
sudo sed '1,/\t#}/s/\t#}/foo/' /etc/nginx/sites-available/default

In [ ]:
sudo sed '0,/\t#}/s/\t#}/\t#}\n\n\tlocation ~ ^\/~(.+?)(\/.*)?$ {\n\t\talias \/home\/$1\/www$2;\n\t\tautoindex on;\n\t}/' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '0,/\t#}/s/\t#}/\t#}\n\n\tlocation ~ ^\/~(.+?)(\/.*)?$ {\n\t\talias \/home\/$1\/www$2;\n\t\tautoindex on;\n\t}/' /etc/nginx/sites-available/default

Jetzt steht der Block drinnen. :-)

## Nginx neu starten

In [ ]:
sudo service nginx restart

Ein Besuch auf http://pi-sense/~pi liefert noch eine 404 Meldung.

In [ ]:
pwd

In [ ]:
mkdir /home/pi/www

Jetzt ist die 404 Meldung weg. Es kann nun eine Webseite erstellt werden.

In [ ]:
cp /var/www/html/index.nginx-debian.html /home/pi/www/index.html

In [ ]:
sudo sed -i 's/Welcome to pi-sense on Respbian!/Welcome to pi Homepage!/g' /home/pi/www/index.html

## PHP und Nginx

In [ ]:
sudo apt-get install --yes php5-fpm

In [ ]:
grep -n php /etc/nginx/sites-available/default

In [ ]:
sed -n '45,52p' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '45,48s/#//' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '50,52s/#//' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i 's/index index.html/index index.php index.html/' /etc/nginx/sites-available/default

In [ ]:
sudo service nginx restart

In [ ]:
echo '<?php echo phpinfo(); ?>' | sudo tee /var/www/html/index.php